In [47]:
import logging
import pickle
import time
from statistics import mean, median

import en_core_web_sm
import gensim.utils
import gensim.models.keyedvectors
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
import string

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from imblearn.under_sampling import RandomUnderSampler
from keras_preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
nltk.download('punkt')

logging.basicConfig(format="%(message)s")
logger = logging.getLogger()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 5 - Model Word2Vec

### 5.1 - Načítanie predspracovaných viet

In [54]:
#Vytvorenie vektorov pre NEUPRAVENÉ VETY = 0
#Vytvorenie vektorov pre UPRAVENÉ VETY (o synonymá) = 1

file = 1
if file == 0:
    file = "sentences"
else:
    file = "sentences_new"

In [55]:
#Načítanie viet
with open("files/"+file, "rb") as fp:   # Unpickling
    sentences = pickle.load(fp)

### 5.2 - Vytvorenie slovných vektorov

In [62]:
dimension_size = 150
window_size = 7
model = 0  # Skipgram = 1, CBOW = 0


embeddings_index = dict()

start = time.time()
#Tréning slovných vektorov
    
word2vec = Word2Vec(
    sentences,
    min_count=1,                      # Ignorovať slová, ktoré sa v korpuse nachádzajú menej ako min_count
    vector_size=dimension_size,       # Veľkosť dimenzie výsledných slovných vektorov
    sg = model,                       # Skipgram = 1, CBOW = 0
    epochs=15, 
    window=window_size,               # Počeť kontextových slov
    workers=6,
    seed=7)
        
words = set(word2vec.wv.index_to_key)        
for word in words:
    embeddings_index[word] = word2vec.wv[word]  
    
end = time.time()

In [63]:
print("Count of words:",len(word2vec.wv),", training time:",(end-start))

Count of words: 25735 , training time: 13.980042219161987


### 5.3 - Uloženie vektorov do BIN súboru

In [64]:
# konvertovanie slovníka do formátu KeyedVectors
kv = KeyedVectors(vector_size=dimension_size)
kv.add_vectors(list(embeddings_index.keys()), list(embeddings_index.values()))

# uloženie slovníka do binárneho súboru
a = "CBOW" if model == 0 else "SKIPGRAM"

kv.save_word2vec_format("files/model_"+a+"_"+"dim_"+str(dimension_size)+"_"+time.strftime("%Y%m%d%H%M")+"_"+file+".bin", binary=True)


attribute count not present in <gensim.models.keyedvectors.KeyedVectors object at 0x0000010941059220>; will store in internal index_to_key order


In [65]:
#Príklad pre podobnosť
word2vec.wv.most_similar("plane")

[('aeroplane', 0.8194882273674011),
 ('activated', 0.5351089239120483),
 ('plunged', 0.49997350573539734),
 ('eerie', 0.47463902831077576),
 ('arrange', 0.45845866203308105),
 ('emitted', 0.4536671042442322),
 ('coincidence', 0.44575566053390503),
 ('refueling', 0.4371525049209595),
 ('layover', 0.43532484769821167),
 ('pretended', 0.4346317648887634)]